In [ ]:
import math
import torch
import time
import random
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.parameter import Parameter
from torch.nn import init
from torch import Tensor
from scipy.special import gamma 
import matplotlib.pyplot as plt

In [ ]:
def get_memory_usage():
    if torch.cuda.is_available():
        return torch.cuda.memory_allocated()/ (1024*1024)
    return 0

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
class Fractional_Order_Matrix_Differential_Solver(torch.autograd.Function):
    @staticmethod
    def forward(ctx,input1,w,b,alpha):
        alpha = torch.tensor(alpha)
        ctx.save_for_backward(input1,w,b,alpha)
        outputs = input1@w + b
        return outputs

    @staticmethod
    def backward(ctx, grad_outputs):
        input1,w,b,alpha = ctx.saved_tensors
        x_fractional, w_fractional = Fractional_Order_Matrix_Differential_Solver.Fractional_Order_Matrix_Differential_Linear(input1,w,b,alpha)   
        x_grad = torch.mm(grad_outputs,x_fractional)
        w_grad = torch.mm(w_fractional,grad_outputs)
        b_grad = grad_outputs.sum(dim=0)
        return x_grad, w_grad, b_grad,None

    @staticmethod
    def Fractional_Order_Matrix_Differential_Linear(x,w,b,alpha):
        #w
        wf = w[:,0].view(1,-1)
        #main
        w_main = torch.mul(x,torch.abs(wf)**(1-alpha)/gamma(2-alpha))
        #partial
        x_rows, x_cols = x.size()
        bias = torch.full((x_rows, x_cols),b[0].item())
        bias = bias.to(device)
        w_partial = torch.mul(torch.mm(x,wf.T).view(-1,1).expand(-1,x_cols) - torch.mul(x,wf) + bias, torch.sign(wf)*torch.abs(wf)**(-alpha)/gamma(1-alpha))
        return w.T, (w_main + w_partial).T

class FLinear(nn.Module):
    
    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: Tensor

    def __init__(self, in_features: int, out_features: int, alpha=0.9, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.weight = Parameter(torch.empty((in_features, out_features), **factory_kwargs))
        if bias:
            self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            init.uniform_(self.bias, -bound, bound)

    def forward(self, x):
        return Fractional_Order_Matrix_Differential_Solver.apply(x, self.weight, self.bias, self.alpha)

    def extra_repr(self) -> str:
        return f"in_features={self.in_features}, out_features={self.out_features}, bias={self.bias is not None}"

In [ ]:
def split(X,y):
    X_train,X_temp,y_train,y_temp = train_test_split(X,y,test_size=0.3,shuffle=False)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.333,shuffle=False)
    return X_train,X_val,X_test,y_train,y_val,y_test

In [ ]:
slide_windows_size = 36  #i.e.,input length
pred_length = 48     #i.e.,prediction lengths,48,96,192,384
stock = 'ETTh1'
df_DJIA = pd.read_csv(r'./data/'+stock+'.csv')
#del df_DJIA['Date']       #DJI
del df_DJIA['date']        #ETT
#scaler = StandardScaler()
scaler = MinMaxScaler(feature_range=(0, 1))

sca_DJIA = scaler.fit_transform(df_DJIA)

In [ ]:
features_j = 6     # DJI is 4, and ETTh1 is 6.
def create_sequences(data, slide_windows_size, pred_length):
    X, y = [], []
    for i in range(len(data) - slide_windows_size - pred_length + 1):
        X.append(data[i:i+slide_windows_size, :])  # sliding window size [seq_len, features]
        y.append(data[i+slide_windows_size:i+slide_windows_size+pred_length, features_j])  
    return np.array(X), np.array(y)

X, y = create_sequences(sca_DJIA, slide_windows_size, pred_length)
X = torch.Tensor(X).to(device)
y = torch.Tensor(y).to(device)

In [ ]:
#Mean Square Error
def MSE(pred,true):
    return np.mean((pred-true)**2)

#Mean Absolute Error
def MAE(pred, true):
    return np.mean(np.abs(pred-true))

In [ ]:
X_train,X_val,X_test,y_train,y_val,y_test = split(X,y)   #7:2:1 

In [ ]:
batch_size = 256
set_seed()
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size=128, output_size=pred_length):   ###DJI:hidden_size=256,ETTh1:hidden_size=128.
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear1 = FLinear(input_size, hidden_size, alpha)       
        #self.linear1 = nn.Linear(input_size, hidden_size)                                  #To calculate the Memory usage on SGD.
        self.linear2 = FLinear(hidden_size, output_size, alpha)      
        #self.linear2 = nn.Linear(hidden_size, output_size)                                 #To calculate the Memory usage on SGD.

    def forward(self, x):
        x = self.flatten(x)  # (batch_size, seq_len*num_features)
        x = self.linear1(x)
        x = self.linear2(x)
        return x

In [ ]:
alpha = 1.0   ####0.7,0.8,0.9,1.0
num_feature = 7      # DJI is 5, and ETTh1 is 7.

set_seed()
model = MLP(input_size=slide_windows_size*num_feature).to(device)

train_loss10 = []    ###
val_loss10 = []      ###

lr =1e-4   
num_epochs = 1500   #1500
best_loss = 10
criterion = nn.MSELoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=lr)
optimizer = torch.optim.SGD(model.parameters(),lr=lr)
#time_start = time.time()
#initial_allocated = get_memory_usage()
for epoch in range(num_epochs):
    model.train()
    loss_sum = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss_sum += loss
        #loss.backward(retain_graph=True)     #The default value of retain_graph is False. only for the first time, to calculate the Memory usage.
        loss.backward()   #The default value of retain_graph is False.
        optimizer.step()
    final_allocated = get_memory_usage() 
    time_end = time.time()
    #print(f"Final allocated memory: {(final_allocated-initial_allocated):.4f} MB")   
    #print(f"Training time: {(time_end-time_start):.4f}s")
    train_loss10.append(loss_sum.cpu().detach().numpy())     ###########
    
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {loss_sum.cpu().detach().numpy():.4f}")
        
    model.eval()
    with torch.no_grad():
        Val_outputs = model(X_val)
        MSE_val = MSE(y_val.cpu().detach().numpy(),Val_outputs.cpu().detach().numpy())
        
        val_loss10.append(MSE_val)   ########################Validation_loss
        
        print(f"Epoch {epoch + 1}/{num_epochs}, Val Loss: {MSE_val:.4f}")
        print('')
        if best_loss > MSE_val:
            best_loss = MSE_val
            #torch.save(model.state_dict(), r'./model/'+stock+'_model_fractional_'+str(alpha).replace(".", "_")+'_'+str(pred_length)+'.pth')   

In [ ]:
plt.figure(figsize=(5,3),dpi=1200)
plt.plot(train_loss07,c='k',label=r'$\alpha=0.7$')
plt.plot(train_loss08,c='y',label=r'$\alpha=0.8$')
plt.plot(train_loss09,c='b',label=r'$\alpha=0.9$')
plt.plot(train_loss10,c='r',label=r'$\alpha=1.0$')
plt.xlim(0,1500)
plt.ylim(0.7,2)
plt.xlabel('$Epochs$')
plt.ylabel('$Loss$')
plt.legend()
#plt.savefig('picture/fig4_c.png',bbox_inches='tight',format='png')
#plt.savefig('picture/fig4_c.svg',bbox_inches='tight',format='svg')
#plt.savefig('picture/fig4_c.pdf',bbox_inches='tight',format='pdf')
plt.show()

In [ ]:
plt.figure(figsize=(5,3),dpi=1200)
plt.plot(val_loss07,c='k',label=r'$\alpha=0.7$')
plt.plot(val_loss08,c='y',label=r'$\alpha=0.8$')
plt.plot(val_loss09,c='b',label=r'$\alpha=0.9$')
plt.plot(val_loss10,c='r',label=r'$\alpha=1.0$')
plt.xlim(0,1500)
plt.ylim(0.02,0.1)
plt.xlabel('$Epochs$')
plt.ylabel('$Loss$')
plt.legend()
#plt.savefig('picture/fig4_d.png',bbox_inches='tight',format='png')
#plt.savefig('picture/fig4_d.svg',bbox_inches='tight',format='svg')
#plt.savefig('picture/fig4_d.pdf',bbox_inches='tight',format='pdf')
plt.show()

In [ ]:
model.load_state_dict(torch.load('./model/'+stock+'_model_fractional_1_0_48.pth'))  ##
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
MSE10 = MSE(y_test.cpu().numpy(),test_outputs.cpu().detach().numpy())
MAE10 = MAE(y_test.cpu().numpy(),test_outputs.cpu().detach().numpy())
print('MSE:',MSE10)
print('MAE:',MAE10)

In [ ]:
model.load_state_dict(torch.load('./model/'+stock+'_model_fractional_0_9_48.pth'))  ##
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
MSE09 = MSE(y_test.cpu().numpy(),test_outputs.cpu().detach().numpy())
MAE09 = MAE(y_test.cpu().numpy(),test_outputs.cpu().detach().numpy())
print('MSE:',MSE09)
print('MAE:',MAE09)

In [ ]:
model.load_state_dict(torch.load('./model/'+stock+'_model_fractional_0_8_48.pth'))  ##
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
MSE08 = MSE(y_test.cpu().numpy(),test_outputs.cpu().detach().numpy())
MAE08 = MAE(y_test.cpu().numpy(),test_outputs.cpu().detach().numpy())
print('MSE:',MSE08)
print('MAE:',MAE08)

In [ ]:
model.load_state_dict(torch.load('./model/'+stock+'_model_fractional_0_7_48.pth'))  ##
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
MSE07 = MSE(y_test.cpu().numpy(),test_outputs.cpu().detach().numpy())
MAE07 = MAE(y_test.cpu().numpy(),test_outputs.cpu().detach().numpy())
print('MSE:',MSE07)
print('MAE:',MAE07)

In [ ]:
MSE_ETTh1 = [0.022396602, 0.022331236, 0.018228948, 0.030933399]    ####MSE_DJI,MAE_DJI
MAE_ETTh1 = [0.11748641, 0.11968101, 0.10541517, 0.13710967]
x = [0.7,0.8,0.9,1.0]

In [ ]:
plt.figure(figsize=(5,3),dpi=1200)

plt.plot(x,MSE_ETTh1,'o-',c='r',label=r'MSE')
plt.plot(x,MAE_ETTh1,'o-',c='b',label=r'MAE')

plt.text(0.71, 0.032396602, f'0.0224', ha='center')
plt.text(0.8, 0.032331236, f'0.0223', ha='center')
plt.text(0.9, 0.028228948, f'0.0182', ha='center')
plt.text(0.99, 0.040933399, f'0.0309', ha='center')

plt.text(0.71, 0.12748641, f'0.1175', ha='center')
plt.text(0.81, 0.12568101, f'0.1197', ha='center')
plt.text(0.9, 0.11541517, f'0.1054', ha='center')
plt.text(0.99, 0.12010967, f'0.1371', ha='center')

plt.xticks(x)
plt.ylabel('$Metrics$')
plt.xlabel(r'$\alpha$')
plt.legend()
#plt.savefig('picture/Fig5_b.png',bbox_inches='tight',format='png')
#plt.savefig('picture/Fig5_b.svg',bbox_inches='tight',format='svg')
#plt.savefig('picture/Fig5_b.pdf',bbox_inches='tight',format='pdf')
plt.show()